In [1]:
%load_ext autotime

In [2]:
import sys
sys.path.append('/scratch/pradap/python-work/enrique/')
# sys.path.append('/Users/pradap/Documents/Research/Python-Package/enrique/')

time: 913 ms


In [3]:
import magellan as mg

time: 43.5 ms


In [4]:
import pandas as pd
import numpy as np

time: 65.4 ms


In [5]:
mg.init_jvm()

True

time: 54.9 ms


In [6]:
wal =  mg.read_csv(mg.get_install_path() + '/datasets/books/walmart.csv',
                    dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                    low_memory=False, key='id')


time: 987 ms


In [7]:
bwk = mg.read_csv(mg.get_install_path() + '/datasets/books/bowker.csv', 
                  dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                  low_memory=False, key='id')


time: 46.7 ms


In [8]:
ab = mg.AttrEquivalenceBlocker()
C = ab.block_tables(wal, bwk, 'isbn', 'isbn', ['title', 'author'], ['title', 'author'])


time: 849 ms


In [9]:
L = mg.read_csv('label_ab_correct_books.csv', ltable=wal, rtable=bwk)


time: 2.99 ms


In [10]:
feat_table = mg.get_features_for_matching(wal, bwk)
f = feat_table.ix[[3,7,18,26, 53]]

time: 2.58 s


In [11]:
G = mg.extract_feat_vecs(L, feat_table=f, attrs_after='gold')

time: 255 ms


In [12]:
dt = mg.DTMatcher(random_state=100)
svm = mg.SVMMatcher()
rf = mg.RFMatcher()
nb = mg.NBMatcher()
lg = mg.LogRegMatcher()

time: 1.9 ms


In [13]:
# impute values
G.fillna(0, inplace=True)

time: 56.2 ms


In [14]:
result = mg.select_matcher([dt, rf, svm, nb, lg], table=G, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
                                  target_attr='gold', metric='precision')

time: 102 ms


In [15]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree_23027393933295695441,<magellan.matcher.dtmatcher.DTMatcher object a...,5,0.875000,0.777778,0.909091,0.916667,0.833333,0.862374
1,RandomForest_69004704933295695441,<magellan.matcher.rfmatcher.RFMatcher object a...,5,0.875000,0.700000,0.909091,0.916667,0.833333,0.846818
2,SVM_29792104933295695441,<magellan.matcher.svmmatcher.SVMMatcher object...,5,0.500000,0.750000,0.888889,0.900000,0.833333,0.774444
3,NaiveBayes_06319314933295695441,<magellan.matcher.nbmatcher.NBMatcher object a...,5,0.875000,0.666667,0.875000,0.900000,1.000000,0.863333
4,LogisticRegression_23692814933295695441,<magellan.matcher.logregmatcher.LogRegMatcher ...,5,0.636364,0.727273,0.800000,0.900000,0.833333,0.779394


time: 15.5 ms


In [16]:
#d = mg.train_test_split(G, train_proportion=0.7)
train = mg.read_csv('train.csv', ltable=wal, rtable=bwk)
test = mg.read_csv('test.csv', ltable=wal, rtable=bwk)


time: 39.4 ms


In [17]:
dt.fit(table=train, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'], target_attr='gold')

time: 37.7 ms


In [18]:
#mg.vis_debug_dt(dt, train, test, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
#                                      target_attr='gold')

time: 34.8 ms


In [19]:
P = dt.predict(table=test, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'], target_attr='predicted', append=True, inplace=False)

time: 43.4 ms


In [20]:
P[['_id', 'title_title_jac_qgm_3_qgm_3', 'gold', 'predicted']]

,_id,title_title_jac_qgm_3_qgm_3,gold,predicted
0,29,1.000000,1,1
1,43,0.421053,0,1
2,61,0.792453,1,1
3,31,0.204819,0,0
4,53,0.861111,1,1
5,19,0.739130,1,1
6,65,0.969697,1,1
7,64,0.936170,1,1
8,21,0.957447,0,1
9,37,0.852941,1,1


time: 87 ms


In [21]:
trigger = mg.MatchTrigger()
trigger.add_cond_rule('title_title_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.8', f)

True

time: 44.6 ms


In [22]:
trigger.add_cond_status(True)
trigger.add_action(0)

True

time: 37.2 ms


In [23]:
Q = trigger.execute(P, 'predicted', inplace=False)

id                                                        3618742
title                                                  Native Son
author                                            Wright, Richard
numAuthors                                                      1
binding                                                 Paperback
publisher                                           Harpercollins
isbn                                                9780060837563
pubYear                                                      2005
pubMonth                                                        8
pubDay                                                          1
editionNum                                                    NaN
pages                                                         512
volume                                                        NaN
lang                                                          ENG
editionDescr                                                  NaN
category  

In [24]:
P[['_id', 'title_title_jac_qgm_3_qgm_3', 'gold', 'predicted']]

,_id,title_title_jac_qgm_3_qgm_3,gold,predicted
0,29,1.000000,1,1
1,43,0.421053,0,1
2,61,0.792453,1,1
3,31,0.204819,0,0
4,53,0.861111,1,1
5,19,0.739130,1,1
6,65,0.969697,1,1
7,64,0.936170,1,1
8,21,0.957447,0,1
9,37,0.852941,1,1


time: 16.1 ms


In [29]:
Q['old_predicted'] = P['predicted']

time: 1.19 ms


In [32]:
Q[['_id', 'title_title_jac_qgm_3_qgm_3', 'gold', 'old_predicted', 'predicted']]

,_id,title_title_jac_qgm_3_qgm_3,gold,old_predicted,predicted
0,29,1.000000,1,1,1
1,43,0.421053,0,1,0
2,61,0.792453,1,1,0
3,31,0.204819,0,0,0
4,53,0.861111,1,1,1
5,19,0.739130,1,1,0
6,65,0.969697,1,1,1
7,64,0.936170,1,1,1
8,21,0.957447,0,1,1
9,37,0.852941,1,1,1


time: 17.9 ms


In [26]:
l_tbl = wal.set_index('id', drop=False)
r_tbl = bwk.set_index('id', drop=False)

time: 450 ms


In [27]:
lt = l_tbl.ix[3618742]
rt = r_tbl.ix[9780060837563]

time: 6.7 ms


In [28]:
P.iat[0, 1]

3618742

time: 34.7 ms
